In [20]:
from keras_image_helper import create_preprocessor
import onnxruntime as ort
import numpy as np
import os

In [4]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

#### Question 1: Output Name

In [15]:
model_path = os.listdir()
model_file = [f for f in model_path if f.endswith('.onnx')][0]
session = ort.InferenceSession(model_file)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [16]:
print("output_name: ", output_name)

output_name:  output


#### Question 2: Size of Image

In [17]:
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
new_image = download_image(url)

In [19]:
target_size = (200, 200)
new_image = prepare_image(new_image, target_size)
print("Prepared image size: ", new_image.size)

Prepared image size:  (200, 200)


#### Question 3: Value in the first pixel, the R channel

In [23]:
x = np.array(new_image, dtype='float32')
print(f"Array shape after conversion: {x.shape}")  

x = x / 255.0

# Apply ImageNet normalization (from homework 8)
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
x = (x - mean) / std


first_pixel_r = x[0, 0, 0]
print(f"First pixel, R channel value: {first_pixel_r}")

Array shape after conversion: (200, 200, 3)
First pixel, R channel value: -1.0732939435925546


#### Question 4: Model Inference

In [24]:
# Transpose from HWC to CHW
x_trans = np.transpose(x, (2, 0, 1))
print(f"Array shape after transpose: {x_trans.shape}")  

Array shape after transpose: (3, 200, 200)


In [26]:
X_batch = np.expand_dims(x_trans, axis=0).astype(np.float32)  # Add batch dimension
print(f"Array shape after adding batch dimension: {X_batch.shape}")

Array shape after adding batch dimension: (1, 3, 200, 200)


In [ ]:
result = session.run([output_name], {input_name: X_batch})
predictions = result[0][0][0]
print(f"Predicted class: {predictions}")

Predicted class: 0.09156627207994461


: 